# IMPORTING ALL LIBRARIES #

In [1]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Model
from keras import Sequential
from keras.utils import Sequence,to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from keras.layers import *
import numpy as np
import pandas as pd
import os
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import pickle
from numpy.linalg import norm
from tqdm import tqdm
from PIL import Image
import io

# DICTIONARY CONTAING ALL CLASSES #

In [3]:
class_dict = {
    'vehicle': {0:'airplane', 1:'automobile', 2:'ship', 3:'truck'},
    'non_vehicle':{0:'bird',1:'cat',2:'deer',3:'dog',4:'frog',5:'horse'}
}

# USING DATA GENERATOR FOR DATA AGUMENTATION FOR TRAIN AND TEST CLASS #

In [56]:
train_datagen = ImageDataGenerator(rescale=1./255,width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

In [57]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [44]:
train_genertaor = train_datagen.flow_from_directory('cifar10/train',target_size=(150,150),batch_size=64,class_mode='binary')
test_genertaor = test_datagen.flow_from_directory('cifar10/test',target_size=(150,150),batch_size=64,class_mode='binary')

Found 50000 images belonging to 2 classes.
Found 10000 images belonging to 2 classes.


# BINARY CLASS MODEL #

In [45]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(150,150,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))


In [46]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 148, 148, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 72, 72, 64)       256       
 hNormalization)                                                 
                                                      

In [47]:
checkpoint_callback = ModelCheckpoint(filepath='models/model_weights_Binary.h5',
                                      monitor='val_accuracy',
                                      verbose=1,
                                      mode='max',
                                      period=2
                                      )

In [48]:
opt = SGD(lr=0.005, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\Home\AppData\Roaming\Python\Python37\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
history = model.fit_generator(train_genertaor,epochs=10,validation_data=test_genertaor, callbacks=[checkpoint_callback])

# Accuracy and Loss plot

In [ ]:

his = model.history.history


plt.plot(his['loss'], label='Training Loss')
plt.plot(his['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# MODEL FOR VEHCLE CLASS #

In [9]:
ve_train_genertaor = train_datagen.flow_from_directory('cifar10/train/vehicle',target_size=(150,150),batch_size=64,class_mode='categorical')
ve_test_genertaor = test_datagen.flow_from_directory('cifar10/test/vehicle',target_size=(150,150),batch_size=64,class_mode='categorical')

Found 20000 images belonging to 4 classes.
Found 4000 images belonging to 4 classes.


In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 batch_normalization_3 (Batc  (None, 148, 148, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 batch_normalization_4 (Batc  (None, 72, 72, 64)       256       
 hNormalization)                                                 
                                                      

In [13]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='models/best_model_weights_cVeh.h5',
                                      monitor='val_accuracy',
                                      verbose=1,
                                      save_best_only=True,
                                      mode='max')

In [ ]:
opt = SGD(lr=0.005, momentum=0.9)
model.compile(optimizer=opt, loss= 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
ve_history = model.fit_generator(ve_train_genertaor,epochs=10,validation_data=ve_test_genertaor, callbacks=[checkpoint_callback])

# Accuracy and LOss pLot

In [ ]:

his = model.history.history

# Plot the metrics
plt.plot(his['loss'], label='Training Loss')
plt.plot(his['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# NON_VEHICLE CLASS MODEL #

In [14]:
nonVe_train_genertaor = train_datagen.flow_from_directory('cifar10/train/non_vehicle', target_size=(150, 150), batch_size=64,
                                                       class_mode='categorical')
nonVe_test_genertaor = test_datagen.flow_from_directory('cifar10/test/non_vehicle', target_size=(150, 150), batch_size=64,
                                                     class_mode='categorical')



Found 30000 images belonging to 6 classes.
Found 6000 images belonging to 6 classes.


In [29]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(150, 150, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

In [30]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 150, 150, 32)      896       
                                                                 
 batch_normalization_10 (Bat  (None, 150, 150, 32)     128       
 chNormalization)                                                
                                                                 
 conv2d_10 (Conv2D)          (None, 150, 150, 32)      9248      
                                                                 
 batch_normalization_11 (Bat  (None, 150, 150, 32)     128       
 chNormalization)                                                
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                      

In [18]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='models/model_weights_cNonVe.h5',
                                      monitor='val_accuracy',
                                      verbose=1,
                                      save_best_only=True,
                                      mode='max')

In [31]:
opt = SGD(lr=0.005, momentum=0.9)
# opt = keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
history = model.fit_generator(nonVe_train_genertaor, epochs=40, validation_data=nonVe_test_genertaor,
                              callbacks=[checkpoint_callback])


Epoch 6: val_accuracy improved from 0.60683 to 0.63633, saving model to models\model_weights_cNonVe.h5
469/469 [==============================] - 4601s 10s/step - loss: 1.0077 - accuracy: 0.6232 - val_loss: 1.4018 - val_accuracy: 0.6363
Epoch 7/10
469/469 [==============================] - ETA: 0s - loss: 0.9701 - accuracy: 0.6365
Epoch 7: val_accuracy improved from 0.63633 to 0.64217, saving model to models\model_weights_cNonVe.h5
469/469 [==============================] - 4415s 9s/step - loss: 0.9701 - accuracy: 0.6365 - val_loss: 1.0947 - val_accuracy: 0.6422
Epoch 8/10
469/469 [==============================] - ETA: 0s - loss: 0.9486 - accuracy: 0.6482
Epoch 8: val_accuracy improved from 0.64217 to 0.65200, saving model to models\model_weights_cNonVe.h5
469/469 [==============================] - 4309s 9s/step - loss: 0.9486 - accuracy: 0.6482 - val_loss: 0.9804 - val_accuracy: 0.6520
Epoch 9/10
469/469 [==============================] - ETA: 0s - loss: 0.9258 - accuracy: 0.6561
Ep

c:\users\home\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/40
469/469 [==============================] - ETA: 0s - loss: 0.8776 - accuracy: 0.6753
Epoch 1: val_accuracy improved from 0.65200 to 0.70050, saving model to models\model_weights_cNonVe.h5
469/469 [==============================] - 4347s 9s/step - loss: 0.8776 - accuracy: 0.6753 - val_loss: 0.8415 - val_accuracy: 0.7005
Epoch 2/40
469/469 [==============================] - ETA: 0s - loss: 0.8673 - accuracy: 0.6795
Epoch 2: val_accuracy did not improve from 0.70050
469/469 [==============================] - 4333s 9s/step - loss: 0.8673 - accuracy: 0.6795 - val_loss: 0.8657 - val_accuracy: 0.6870
Epoch 3/40
469/469 [==============================] - ETA: 0s - loss: 0.8461 - accuracy: 0.6895
Epoch 3: val_accuracy did not improve from 0.70050
469/469 [==============================] - 4341s 9s/step - loss: 0.8461 - accuracy: 0.6895 - val_loss: 0.9160 - val_accuracy: 0.6937
Epoch 4/40
469/469 [==============================] - ETA: 0s - loss: 0.8302 - accuracy: 0.6942
Epoch 4: val

# Accuracy and Loss Plot

In [ ]:

his = model.history.history

# Plot the metrics
plt.plot(his['loss'], label='Training Loss')
plt.plot(his['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# IMAGE SEARCH MODEL #

In [12]:
base_path = 'net_image_dataset'

images_path = {}
for folders in os.listdir(base_path):
    for fol in os.listdir(f'{base_path}/{folders}'):
        print(fol)
        break
        list = []
        for file in os.listdir(f'{base_path}/{folders}/{fol}'):

            list.append((f'{base_path}/{folders}/{fol}/{file}'))
        images_path[f'{folders}/{fol}'] = list

frog
airplane


In [3]:
images_path.keys()

dict_keys(['non_vehicle/frog', 'vehicle/airplane', 'vehicle/car', 'vehicle/truck'])

In [4]:
model = ResNet50(weights='imagenet', include_top=False, input_shape=(32,32,3))
model.trainable = False

model = keras.Sequential([
    model,
    GlobalMaxPool2D()
])

In [5]:
def extract_features(img_path,model):
    img = image.load_img(img_path, target_size=(32, 32))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    pre_model = model.predict(preprocessed_img).flatten()
    normalized_result = pre_model/norm(pre_model)
    return normalized_result


In [10]:
images_path

{'non_vehicle/frog': ['net_image_dataset/non_vehicle/frog/istockphoto-1134798796-612x612.jpg',
  'net_image_dataset/non_vehicle/frog/istockphoto-1347313672-612x612.jpg'],
 'vehicle/airplane': ['net_image_dataset/vehicle/airplane/14386209.jpeg',
  'net_image_dataset/vehicle/airplane/40907.jpeg'],
 'vehicle/car': ['net_image_dataset/vehicle/car/1000558.jpeg',
  'net_image_dataset/vehicle/car/1003868.jpeg',
  'net_image_dataset/vehicle/car/1030649.jpeg',
  'net_image_dataset/vehicle/car/1034662.jpeg',
  'net_image_dataset/vehicle/car/1090932.jpeg',
  'net_image_dataset/vehicle/car/1125329.jpeg',
  'net_image_dataset/vehicle/car/13061205.jpeg',
  'net_image_dataset/vehicle/car/13356766.jpeg',
  'net_image_dataset/vehicle/car/14273496.jpeg',
  'net_image_dataset/vehicle/car/14277864.jpeg',
  'net_image_dataset/vehicle/car/1456011.jpeg',
  'net_image_dataset/vehicle/car/15377954.jpeg',
  'net_image_dataset/vehicle/car/15750673.jpeg',
  'net_image_dataset/vehicle/car/1600143.jpeg',
  'net_ima

In [6]:
features = {}
for file in tqdm(images_path):
    list = []
    for data in images_path[file]:
        list.append(extract_features(data, model))
    features[file] = list

  0%|          | 0/4 [00:00<?, ?it/s]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 104ms/step


 25%|██▌       | 1/4 [00:06<00:19,  6.57s/it]

1/1 [==============================] - 0s 128ms/step


 50%|█████     | 2/4 [00:07<00:06,  3.05s/it]

1/1 [==============================] - 0s 65ms/step


 75%|███████▌  | 3/4 [01:04<00:27, 27.87s/it]

1/1 [==============================] - 0s 63ms/step


100%|██████████| 4/4 [02:36<00:00, 39.18s/it]


In [7]:
for data in features:
    name = data.replace('/','_')
    pickle.dump(features[data], open(f'features_{name}.pkl', 'wb'))
    pickle.dump(images_path[data], open(f'filenames_{name}.pkl', 'wb'))


# PIPELINE

In [11]:
def pipeline(test_img):
    test_img = Image.open(test_img)
    b_model = keras.models.load_model('models/best_model_weights_Binary.h5')
    cnn_model_veh = keras.models.load_model('models/model_weights_cVeh.h5')
    cnn_model_nonVeh = keras.models.load_model('models/model_weights_cNonVe.h5')

    img = test_img.resize((150,150))
    img = np.expand_dims(img, axis=0)
    pred_img = b_model.predict(img)

    # mul_img = test_img.resize((32,32))
    # mul_img = np.expand_dims(mul_img, axis=0)

    predictions = np.array([])
    if pred_img < 0.5  and pred_img >= 0:
        predictions = cnn_model_nonVeh.predict(test_img)
    elif(pred_img > 0.5):
        predictions = cnn_model_veh.predict(test_img)

    predicted_class = np.argmax(predictions,axis=-1)
    prediction = ""
    if pred_img < 0.5:
        prediction = 'non_vehicle/'
        prediction = prediction + class_dict['non_vehicle'][predicted_class[0]]
    elif pred_img > 0.5:
        prediction = 'vehicle/'
        prediction = prediction + class_dict['vehicle'][predicted_class[0]]

    t_img = test_img.resize((32,32))
    img_array = image.img_to_array(t_img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    pre_model = model.predict(preprocessed_img).flatten()
    normalized_result = pre_model/norm(pre_model)

    features_list = {}
    filenames = {}

    for files in images_path:
        name = files.replace('/','_')
        features_list[files] = np.array(pd.read_pickle(f'features_{name}.pkl'))
        filenames[files] = pd.read_pickle(open(f'filenames_{name}.pkl','rb'))


    neighbors = NearestNeighbors(n_neighbors = 7,algorithm='brute',metric='euclidean')
    neighbors.fit(features_list[prediction])

    distances, indices = neighbors.kneighbors(normalized_result.reshape(1,-1))
    file_list = filenames[prediction]

    list = []
    for i in indices[0]:
        list.append(file_list[i])

    return prediction.split('/')[0],prediction.split('/')[1],list

In [12]:
pipeline('uploads/image.jpg')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 1s 1s/step


ValueError: Failed to find data adapter that can handle input: <class 'PIL.JpegImagePlugin.JpegImageFile'>, <class 'NoneType'>